# Simulations of Yeast

This notebook is based on:

https://github.com/BenjaSanchez/notebooks/blob/8895b5293dadc9fe3d1c785d28523bcf8fc8c65f/e_coli_simulations/notebook.ipynb

And is used to transform the model bla of to comply with the standards required for upload on ddecaf.


Jonas Sindlinger, 2020-02-07

## 1. Loading models

* ec-yeast model: https://raw.githubusercontent.com/SysBioChalmers/ecModels/master/ecYeastGEM/model/ecYeastGEM.xml


In [4]:
import cobra
import os
import wget
import pandas as pd
import ssl
import math


# Enzyme-constrained model:
ssl._create_default_https_context = ssl._create_unverified_context
wget.download("https://raw.githubusercontent.com/SysBioChalmers/ecModels/master/ecYeastGEM/model/ecYeastGEM.xml", "ecYeastGEM.xml", bar=False)
ecModel = cobra.io.read_sbml_model("ecYeastGEM.xml")
os.remove("ecYeastGEM.xml")
ecModel.summary()

In [5]:
# loading of translation files for mets and reactions 
wget.download("https://raw.githubusercontent.com/SysBioChalmers/yeast-GEM/3b83df2a442d7b874a4f1d0e19e7eeeca6142f1d/ComplementaryData/modelCuration/globalMetDictionary.csv", "globalMetDictionary.csv", bar=False)
pd_metId2name = pd.read_csv("globalMetDictionary.csv", header=None)
pd_metId2name.columns = ["id", "BiGG"]
os.remove("globalMetDictionary.csv")


# creating dictionary from pandas df
dict_metId2name = pd_metId2name.set_index("id").to_dict()["BiGG"]

### 2. Set Metabolite names to BiGG format 

In [6]:
%%capture
# translation of ids to BiGG
[setattr(met,'id',dict_metId2name[met.id]) for met in ecModel.metabolites if met.id in dict_metId2name]

In [7]:
ecModel.summary()

### 2 Fixing the enzyme-constrained model

cobrapy cannot handle upper bounds = `Inf`, therefore we need to replace them with `1000` (standard in the field):

In [8]:
for reaction in ecModel.reactions:
    if math.isinf(reaction.upper_bound):
        reaction.upper_bound = 1000

ecModel.optimize()



,fluxes,reduced_costs
r_0006,0.022002,0.000000
r_0070,0.000000,0.000000
r_0094,0.000000,0.000000
r_0099,0.000000,0.000000
r_0200,0.000000,-0.041053
...,...,...
prot_Q99190_exchange,0.000000,-0.000000
prot_Q99258_exchange,0.000000,-0.000000
prot_Q99288_exchange,0.000000,-0.000000
prot_Q99312_exchange,0.000000,0.000000


### 3. Fixing metabolite name suffixes

In [9]:
# Metabolite ids: remove the trailing "[comp]" (if any):
for metabolite in ecModel.metabolites:
    trail = "[" + metabolite.compartment + "]"
    if metabolite.id.endswith(trail):
        metabolite.id = metabolite.id.split(trail)[0] + "_" + metabolite.compartment



In [10]:
ecModel.summary()

### 4. Export model

In [11]:
#Save model as .xml & .json:
cobra.io.save_json_model(ecModel, "models/ecYeastGEM.json")